In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from copy import deepcopy

import k3d
import yaml
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm
import json

import warnings

from joblib import Parallel, delayed
from collections import defaultdict

In [23]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '10'
os.environ['OMP_NUM_THREADS'] = '10'
os.environ['MKL_NUM_THREADS'] = '10'

In [24]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 0.5

            colors = k3d.helpers.map_colors(
                samples_distances, k3d.colormaps.matplotlib_color_maps.coolwarm_r, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [25]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [26]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_point_patches
from sharpf.data.mesh_nbhoods import NBHOOD_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.data.point_samplers import SAMPLER_BY_TYPE
from sharpf.utils.abc_utils.abc.feature_utils import compute_features_nbhood, remove_boundary_features, get_curves_extents
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.geometry import mean_mmd
import sharpf.data.data_smells as smells


In [574]:
with ABCChunk(['/data/abc/abc_0050_obj_v00.7z', '/data/abc/abc_0050_feat_v00.7z']) as data_holder:
    item = data_holder.get('00502539_6e851dae74079e807f2dbcf4_012')

In [575]:
item.item_id

'00502539_6e851dae74079e807f2dbcf4_012'

In [576]:
# mesh, vertex_normals, vertex_normal_indices = trimesh_load(item.obj)
mesh, _, _ = trimesh_load(item.obj)

features = yaml.load(item.feat, Loader=yaml.Loader)

In [577]:
mesh2 = mesh.copy()
mesh2._validate = True

In [578]:
mesh2.process()

<trimesh.Trimesh(vertices.shape=(21382, 3), faces.shape=(42764, 3))>

In [579]:
mesh

<trimesh.Trimesh(vertices.shape=(21382, 3), faces.shape=(42764, 3))>

In [580]:
mesh.is_watertight

True

In [581]:
for i in range(len(features['surfaces'])):
    print(i, np.array(np.unique(mesh.faces[features['surfaces'][i]['face_indices']]) == 
                      np.sort(features['surfaces'][i]['vert_indices'])).all()
         )

0 True
1 False
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 False
33 False
34 False
35 False
36 False


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [582]:
with open('/code/scripts/data_scripts/configs/pointcloud_datasets/dataset_config_high_res_clean.json') as config_file:
    config = json.load(config_file)

config['neighbourhood']['max_patches_per_mesh'] = 10000
# config['sampling']['resolution_3d'] = 0.05
# config['smell_deviating_resolution'] = {
#     'resolution_3d': 0.05,
#     'resolution_deviation_tolerance': 0.05 / 2
# }
config

{'shape_fabrication_extent': 10.0,
 'short_curve_quantile': 0.25,
 'base_n_points_per_short_curve': 8,
 'base_resolution_3d': 0.125,
 'neighbourhood': {'type': 'random_euclidean_sphere',
  'max_patches_per_mesh': 10000,
  'n_vertices': None,
  'centroid': None,
  'centroid_mode': 'poisson_disk',
  'radius_base': 10.0,
  'radius_delta': 0.0,
  'geodesic_patches': True,
  'radius_scale_mode': 'no_scale'},
 'sampling': {'type': 'poisson_disk',
  'n_points': 4096,
  'resolution_3d': 0.02,
  'make_n_points': 'crop_center'},
 'noise': {'type': 'many_noisers',
  'subtype': 'isotropic_gaussian',
  'scale': [0.0]},
 'annotation': {'type': 'surface_based_aabb', 'distance_upper_bound': 1.0},
 'smell_coarse_surfaces_by_num_edges': {'num_edges_threshold': 8},
 'smell_coarse_surfaces_by_angles': {'max_angle_threshold_degrees': 10.0},
 'smell_deviating_resolution': {'resolution_3d': 0.02,
  'resolution_deviation_tolerance': 0.01},
 'smell_sharpness_discontinuities': {},
 'smell_bad_face_sampling': {'

In [583]:
# shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
# base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
# base_resolution_3d = config.get('base_resolution_3d', LOW_RES)
# short_curve_quantile = config.get('short_curve_quantile', 0.05)
shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = 8
base_resolution_3d = 0.15
short_curve_quantile = 0.25

nbhood_extractor = load_func_from_config(NBHOOD_BY_TYPE, config['neighbourhood'])
sampler = load_func_from_config(SAMPLER_BY_TYPE, config['sampling'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])


smell_coarse_surfaces_by_num_edges = smells.SmellCoarseSurfacesByNumEdges.from_config(config['smell_coarse_surfaces_by_num_edges'])
smell_coarse_surfaces_by_angles = smells.SmellCoarseSurfacesByAngles.from_config(config['smell_coarse_surfaces_by_angles'])
smell_deviating_resolution = smells.SmellDeviatingResolution.from_config(config['smell_deviating_resolution'])
smell_sharpness_discontinuities = smells.SmellSharpnessDiscontinuities.from_config(config['smell_sharpness_discontinuities'])
smell_bad_face_sampling = smells.SmellBadFaceSampling.from_config(config['smell_bad_face_sampling'])


In [584]:

def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    mesh = mesh.apply_scale(least_len_mm / least_len)

    return mesh


In [585]:
mesh = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                      short_curve_quantile=short_curve_quantile,
                      n_points_per_short_curve=base_n_points_per_short_curve)

In [586]:
np.max(mesh.bounding_box.extents)

16.79999976480001

In [587]:
working_range = np.array([1000, 600, 300])
resolutions_3d = np.array([0.25, 0.15, 0.06])
std_3d = np.array([0.12, 0.06, 0.04])

In [588]:
k_wr_r3d = 0.5 * (working_range[1] - working_range[0]) / (resolutions_3d[1] - resolutions_3d[0]) + \
0.5 * (working_range[2] - working_range[1]) / (resolutions_3d[2] - resolutions_3d[1]) 

In [589]:
k_wr_e3d = 0.5 * (working_range[1] - working_range[0]) / (std_3d[1] - std_3d[0]) + \
    0.5 * (working_range[2] - working_range[1]) / (std_3d[2] - std_3d[1]) 

In [590]:
target_resolution_3d = 1.0

In [591]:
mesh = scale_mesh(mesh, features, shape_fabrication_extent, target_resolution_3d,
                      short_curve_quantile=short_curve_quantile,
                      n_points_per_short_curve=base_n_points_per_short_curve)

In [592]:
np.max(mesh.bounding_box.extents)

111.99999843200006

In [593]:
import trimesh

In [594]:
stl_mesh = trimesh.exchange.export.export_stl(mesh)

In [595]:
filename = '{item_id}__{size:3.1f}mm.stl'.format(
    item_id=item.item_id,
    size=np.max(mesh.bounding_box.extents))
with open(filename, 'wb') as f:
    f.write(stl_mesh)

In [55]:
smells.SmellMeshSelfIntersections().run(mesh)

False

In [58]:
# surf_vert_indices = np.array(features['surfaces'][0]['vert_indices'])
# surf_verts = mesh.vertices[surf_vert_indices]
# surf_vert_indices2 = np.array(features['surfaces'][0]['vert_indices'])
# surf_verts2 = mesh.vertices[surf_vert_indices]



display_sharpness(
    mesh, plot_meshvert=True, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, 
    sharpvert_psize=1. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples=None, samples_psize=1. * sampler.resolution_3d,
)

Output()

Having surface area $S$, sampling resolution $r$ (= mean distance between points), and considering each point as occupying a surface $\pi r^2 / 4$, we have the number of points as:
$$
n = S / \frac {\pi r^2} {4}
$$

In [438]:
n_points = np.ceil(mesh.area / (np.pi * 0.02 ** 2 / 4)).astype(int)
mesh.area, n_points

(2.062002725243821, 6564)

In [95]:
import igl
import trimesh

def _make_dense_mesh(mesh, n_desired_points, extra_points_factor=10, point_split_factor=4):
    # Intuition: take 10x the number of needed n_points,
    # keep in mind that each call to `igl.upsample` generates 4x the points,
    # then compute the upsampling factor K from the relation:
    # 4^K n = 10 n_points
    upsampling_factor = np.ceil(
        np.log(n_desired_points * extra_points_factor / len(mesh.vertices)) /
        np.log(point_split_factor)
    ).astype(int)

    # Generate very dense subdivision samples on the mesh (v, f, n)
    # for _ in range(upsampling_factor):
    #     mesh = mesh.subdivide()
    dense_points, dense_faces = igl.upsample(mesh.vertices, mesh.faces, upsampling_factor)

    # compute vertex normals by pushing to trimesh
    dense_mesh = trimesh.base.Trimesh(vertices=dense_points, faces=dense_faces, process=False, validate=False)
    return dense_mesh

In [96]:
import point_cloud_utils as pcu

dense_mesh = _make_dense_mesh(
    mesh,
    n_desired_points=n_points,
    extra_points_factor=10,
    point_split_factor=4
)

dense_points = np.array(dense_mesh.vertices, order='C')
dense_normals = np.array(dense_mesh.vertex_normals, order='C')
dense_faces = np.array(dense_mesh.faces)

In [ ]:
# Downsample v_dense to be from a blue noise distribution:
#
# `points` is a downsampled version of `dense_points` where points are separated by approximately
# `radius` distance, use_geodesic_distance indicates that the distance should be measured on the mesh.
#
# `normals` are the corresponding normals of `points`

# require a little bit extra points as PDS get you sometimes fewer than requested
required_points = int(1.1 * n_points)
points, normals = pcu.sample_mesh_poisson_disk(
    dense_points, dense_faces, dense_normals,
    required_points, radius=sampler.resolution_3d, use_geodesic_distance=True)


In [ ]:
display_sharpness(
    mesh, plot_meshvert=False, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, 
    sharpvert_psize=1. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples=points, samples_psize=1. * sampler.resolution_3d,
)

In [50]:
full_model_resolution_discount = 4.0
nbhood_extractor.radius_base = np.sqrt(sampler.n_points) * 0.5 * sampler.resolution_3d / full_model_resolution_discount

nbhood_extractor.index(mesh)

full_model_resolution_discount = 1.0
nbhood_extractor.radius_base = np.sqrt(sampler.n_points) * 0.5 * sampler.resolution_3d / full_model_resolution_discount

In [51]:
len(nbhood_extractor.centroids_cache)

347

In [52]:
has_smell_mismatching_surface_annotation = any([
    np.array(np.unique(mesh.faces[surface['face_indices']]) != np.sort(surface['vert_indices'])).all()
    for surface in features['surfaces']
])

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
LARGEST_PROCESSABLE_MESH_VERTICES = 20000

nbhood_extractor.current_patch_idx = 0

# extract neighbourhood
try:
    nbhood, mesh_vertex_indexes, mesh_face_indexes, scaler = nbhood_extractor.get_nbhood()
    if len(nbhood.vertices) > LARGEST_PROCESSABLE_MESH_VERTICES:
        raise DataGenerationException('Too large number of vertices in crop: {}'.format(len(nbhood.vertices)))
except DataGenerationException as e:
    eprint_t(str(e))

In [62]:
display_sharpness(
    nbhood, plot_meshvert=False, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, 
    sharpvert_psize=1. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
#     samples=points, samples_psize=1. * sampler.resolution_3d,
)

Output()

In [63]:
centroid = nbhood_extractor.centroid

has_smell_coarse_surfaces_by_num_edges = smell_coarse_surfaces_by_num_edges.run(mesh, mesh_face_indexes, features)
has_smell_coarse_surfaces_by_angles = smell_coarse_surfaces_by_angles.run(mesh, mesh_face_indexes, features)

# create annotations: condition the features onto the nbhood
nbhood_features = compute_features_nbhood(mesh, features, mesh_face_indexes,
                                          mesh_vertex_indexes=mesh_vertex_indexes)

# remove vertices lying on the boundary (sharp edges found in 1 face only)
nbhood_features = remove_boundary_features(nbhood, nbhood_features, how='edges')



In [69]:
from scipy.spatial import cKDTree

In [70]:
tree = cKDTree(points, leafsize=100)


In [71]:
_, indexes = tree.query(centroid, k=4096)

In [73]:
patch_points, patch_normals = points[indexes], normals[indexes]

In [74]:
display_sharpness(
    nbhood, plot_meshvert=False, meshvert_psize=sampler.resolution_3d,
    sharp_vert=None, 
    sharpvert_psize=1. * sampler.resolution_3d,
#     sharp_vert=mesh.vertices[boundary_curves[1]['vert_indices']],
    samples=patch_points, samples_psize=1. * sampler.resolution_3d,
)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [ ]:
LARGEST_PROCESSABLE_MESH_VERTICES = 20000


def compute_patches(patch_idx, mesh, features,
                    nbhood_extractor, sampler, noiser,
                    smell_coarse_surfaces_by_num_edges,
                    smell_coarse_surfaces_by_angles,
                    smell_deviating_resolution,
                    smell_bad_face_sampling):
    
    global LARGEST_PROCESSABLE_MESH_VERTICES

    patches_by_config = defaultdict(list)

    nbhood_extractor.current_patch_idx = patch_idx

    # extract neighbourhood
    try:
        nbhood, mesh_vertex_indexes, mesh_face_indexes, scaler = nbhood_extractor.get_nbhood()
        if len(nbhood.vertices) > LARGEST_PROCESSABLE_MESH_VERTICES:
            raise DataGenerationException('Too large number of vertices in crop: {}'.format(len(nbhood.vertices)))
    except DataGenerationException as e:
        eprint_t(str(e))
        return None
    centroid = nbhood_extractor.centroid

    has_smell_coarse_surfaces_by_num_edges = smell_coarse_surfaces_by_num_edges.run(mesh, mesh_face_indexes, features)
    has_smell_coarse_surfaces_by_angles = smell_coarse_surfaces_by_angles.run(mesh, mesh_face_indexes, features)

    # create annotations: condition the features onto the nbhood
    nbhood_features = compute_features_nbhood(mesh, features, mesh_face_indexes,
                                              mesh_vertex_indexes=mesh_vertex_indexes)

    # remove vertices lying on the boundary (sharp edges found in 1 face only)
    nbhood_features = remove_boundary_features(nbhood, nbhood_features, how='edges')

    # sample the neighbourhood to form a point patch
    try:
        points, normals = sampler.sample(nbhood, centroid=nbhood_extractor.centroid)
    except DataGenerationException as e:
        eprint_t(str(e))
        return None

    has_smell_deviating_resolution = smell_deviating_resolution.run(points)
    has_smell_bad_face_sampling = smell_bad_face_sampling.run(nbhood, points)

    # create a noisy sample
    for configuration, noisy_points in noiser.make_noise(points, normals):
        num_sharp_curves = len([curve for curve in nbhood_features['curves'] if curve['sharp']])
        num_surfaces = len(nbhood_features['surfaces'])
        patch_info = {
            'points': np.array(noisy_points).astype(np.float64),
            'normals': np.array(normals).astype(np.float64),
            # 'distances': np.array(distances).astype(np.float64),
            # 'directions': np.array(directions).astype(np.float64),
#             'item_id': item_id,
            'orig_vert_indices': np.array(mesh_vertex_indexes).astype(np.int32),
            'orig_face_indexes': np.array(mesh_face_indexes).astype(np.int32),
            # 'has_sharp': has_sharp,
            'num_sharp_curves': num_sharp_curves,
            'num_surfaces': num_surfaces,
            'has_smell_coarse_surfaces_by_num_faces': has_smell_coarse_surfaces_by_num_edges,
            'has_smell_coarse_surfaces_by_angles': has_smell_coarse_surfaces_by_angles,
            'has_smell_deviating_resolution': has_smell_deviating_resolution,
            # 'has_smell_sharpness_discontinuities': has_smell_sharpness_discontinuities,
            'has_smell_bad_face_sampling': has_smell_bad_face_sampling,
            # 'has_smell_mismatching_surface_annotation': has_smell_mismatching_surface_annotation,
            'nbhood': nbhood,
            'nbhood_features': nbhood_features,
            'centroid': centroid,
            'nbhood_radius': nbhood_extractor.radius_base,
        }
        config_name = configuration.get('name')
        patches_by_config[config_name].append(patch_info)

    return patches_by_config


In [ ]:
parallel = Parallel(n_jobs=36, backend='multiprocessing', verbose=100)
delayed_iterable = (delayed(compute_patches)(
        patch_idx, mesh, features,
        nbhood_extractor, sampler, noiser,
        smell_coarse_surfaces_by_num_edges,
        smell_coarse_surfaces_by_angles,
        smell_deviating_resolution,
        smell_bad_face_sampling)
        for patch_idx in range(nbhood_extractor.n_patches_per_mesh))
result = parallel(delayed_iterable)

In [ ]:
patches_by_config = defaultdict(list)

for r in result:
    for config_name, patch_infos in r.items():
        for info in patch_infos:
            info.update({
                'item_id': item.item_id,
                'has_smell_mismatching_surface_annotation': has_smell_mismatching_surface_annotation,
            })
        patches_by_config[config_name].extend(patch_infos)

In [ ]:
patches = patches_by_config[0.0]

In [ ]:
whole_model_points = np.concatenate([
    patch['points']
    for patch in patches
])
whole_model_points.shape

In [ ]:
whole_model_points_norm_sq = np.linalg.norm(whole_model_points, axis=1) ** 2

In [ ]:
idx = 14
centroid = patches[idx]['centroid']

In [ ]:
%time distance_to_centroid = np.sqrt(np.square(whole_model_points - centroid).sum(axis=1))

In [ ]:
%%time 
distance_to_centroid2 = np.sqrt(
    (whole_model_points_norm_sq 
     - 2 * np.dot(whole_model_points, centroid) 
     + np.linalg.norm(centroid) ** 2)
)

In [ ]:
%time indexes = np.where(distance_to_centroid < nbhood_extractor.radius_base)[0]

In [ ]:
for idx in tqdm(range(len(patches))):
    centroid = patches[idx]['centroid']
    distance_to_centroid = np.sqrt(
        whole_model_points_norm_sq 
         - 2 * np.dot(whole_model_points, centroid) 
         + np.linalg.norm(centroid) ** 2
    )
    indexes = np.where(distance_to_centroid < nbhood_extractor.radius_base)[0]
    noisy_points = whole_model_points[indexes]

In [ ]:
def compute_annotation_nonlocal(patch, whole_model_points, config):
    annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])
    smell_sharpness_discontinuities = smells.SmellSharpnessDiscontinuities.from_config(
        config['smell_sharpness_discontinuities'])

    whole_model_points_norm_sq = np.linalg.norm(whole_model_points, axis=1) ** 2
    centroid = patch['centroid']
    distance_to_centroid = np.sqrt(
        whole_model_points_norm_sq 
         - 2 * np.dot(whole_model_points, centroid) 
         + np.linalg.norm(centroid) ** 2
    )
    indexes = np.where(distance_to_centroid < nbhood_extractor.radius_base)[0]
    noisy_points = whole_model_points[indexes]    
    
    nbhood = patch['nbhood']
    nbhood_features = patch['nbhood_features']
    
    try:
        distances, directions, has_sharp = annotator.annotate(nbhood, nbhood_features, noisy_points)
    except DataGenerationException as e:
        eprint_t(str(e))
        return [None] * 5
    has_smell_sharpness_discontinuities = smell_sharpness_discontinuities.run(noisy_points, distances)

    return distances, directions, has_sharp, indexes, has_smell_sharpness_discontinuities

In [ ]:
from joblib import Parallel, delayed

parallel = Parallel(n_jobs=20, backend='multiprocessing', verbose=100)
delayed_iterable = (delayed(compute_annotation_nonlocal)(patch, whole_model_points, config) for patch in patches)
result = parallel(delayed_iterable)


In [ ]:
whole_model_distances = np.ones(len(whole_model_points)) * np.inf
whole_model_directions = np.ones( (len(whole_model_points), 3) ) * np.inf

In [ ]:
for i, (distances, directions, has_sharp, indexes, has_smell_sharpness_discontinuities) in tqdm(enumerate(result)):
    assign_mask = whole_model_distances[indexes] > distances
    whole_model_distances[indexes[assign_mask]] = distances[assign_mask]
    whole_model_directions[indexes[assign_mask]] = directions[assign_mask]

In [ ]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=whole_model_points, samples_distances=whole_model_distances,
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=None)

In [ ]:
min_idx = whole_model_distances.argmin(axis=0)

In [ ]:
min_idx

In [ ]:
whole_patches = []

for i, (patch, relabeled) in tqdm(enumerate(zip(patches, result))):
    distances, directions, has_sharp, indexes, has_smell_sharpness_discontinuities = relabeled
    
    whole_patch = deepcopy(patch)
    i1, i2 = i * sampler.n_points, (i + 1) * sampler.n_points
    whole_patch['distances'] = whole_model_distances[i1:i2]
    whole_patch['directions'] = whole_model_directions[i1:i2, :]
    nbhood_features = patch['nbhood_features']
    whole_patch['has_sharp'] = any(curve['sharp'] for curve in nbhood_features['curves'])
    whole_patch['num_sharp_curves'] = len([curve for curve in nbhood_features['curves'] if curve['sharp']])
    whole_patch['num_surfaces'] = len(nbhood_features['surfaces'])
    whole_patch['has_smell_mismatching_surface_annotation'] = False
    whole_patch['has_smell_sharpness_discontinuities'] = has_smell_sharpness_discontinuities
    whole_patches.append(whole_patch)

In [ ]:
idx = np.random.randint(len(patches))

patch = whole_patches[idx]

display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=2 * sampler.resolution_3d,
                  samples=patch['points'], samples_distances=patch['distances'],
                  samples_color=0x0000ff, samples_psize=sampler.resolution_3d,
                  directions=patch['directions'])

In [ ]:
from sharpf.data.datasets.sharpf_io import save_point_patches


In [ ]:
save_point_patches(whole_patches, '/logs/whole_{}.hdf5'.format(item.item_id))

In [7]:
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes
import sharpf.utils.abc_utils.hdf5.io_struct as io
from sharpf.data.datasets.sharpf_io import PointCloudIO

In [8]:
PointCloudIO = io.HDF5IO({
    'points': io.Float64('points'),
    'normals': io.Float64('normals'),
    'distances': io.Float64('distances'),
    'directions': io.Float64('directions'),
    'item_id': io.AsciiString('item_id'),
    'orig_vert_indices': io.VarInt32('orig_vert_indices'),
    'orig_face_indexes': io.VarInt32('orig_face_indexes'),
    'has_sharp': io.Bool('has_sharp'),
    'num_sharp_curves': io.Int8('num_sharp_curves'),
    'num_surfaces': io.Int8('num_surfaces'),
    'has_smell_coarse_surfaces_by_num_faces': io.Bool('has_smell_coarse_surfaces_by_num_faces'),
    'has_smell_coarse_surfaces_by_angles': io.Bool('has_smell_coarse_surfaces_by_angles'),
    'has_smell_deviating_resolution': io.Bool('has_smell_deviating_resolution'),
    'has_smell_sharpness_discontinuities': io.Bool('has_smell_sharpness_discontinuities'),
    'has_smell_bad_face_sampling': io.Bool('has_smell_bad_face_sampling'),
    'has_smell_mismatching_surface_annotation': io.Bool('has_smell_mismatching_surface_annotation'),
# 'voronoi': io.Float64('voronoi'),
# 'normals_estimation_10': io.Float64('normals_estimation_10'),
# 'normals_estimation_100': io.Float64('normals_estimation_100'),

},
len_label='has_sharp',
compression='lzf')

In [20]:
dataset = Hdf5File('/logs/abc_0000_00007710_5e85263979fb44d18725b575_008.hdf5',
                   io=PointCloudIO,
                   preload=PreloadTypes.LAZY,
                   labels='*')

In [21]:
whole_model_points = np.concatenate([
    patch['points']
    for patch in dataset
])
whole_model_points.shape

(3067904, 3)

In [22]:
whole_model_directions = np.concatenate([
    patch['directions']
    for patch in dataset
])
whole_model_directions.shape

(3067904, 3)

In [23]:
whole_model_distances = np.concatenate([
    patch['distances']
    for patch in dataset
])
whole_model_distances.shape

(3067904,)

In [24]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  sharp_vert=None, sharpvert_psize=0.02,
                  samples=whole_model_points, samples_distances=whole_model_distances,
                  samples_color=0x0000ff, samples_psize=0.02,
                  directions=None)

Output()